In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/ihpar/ddi/main/LAB2%20-%20TF-IDF/filmler_clean.csv")

In [3]:
df.head()

,film adi,ozet ve detaylar,tur,oyuncular,yonetmen,vizyon tarihi,sure,clean
0,Bergen,Arabeskin unutulmayan kadın seslerinden biri o...,"['Dram', 'Biyografik']","['Farah Zeynep Abdullah', 'Erdal Beşikçioğlu',...","['Mehmet Binay', 'Caner Alper']",4 Mart 2022,2s 26dk,arabesk unutmak kadın ses o Bergen inmek hayat...
1,Kirpi Sonic 2,Çocuk filmi \n\n\nKirpi Sonic filminin devam h...,"['Animasyon', 'Aile', 'Macera']","['Jim Carrey', 'James Marsden', None]",['Jeff Fowler'],8 Nisan 2022,2s 2dk,çocuk film kirpi Sonic film deva halka o yapım...
2,Fantastik Canavarlar: Dumbledore'un Sırları,"Karanlık büyücü Gellert Grindelwald, büyücülük...","['Fantastik', 'Macera']","['Eddie Redmayne', 'Jude Law', 'Mads Mikkelsen']",['David Yates'],15 Nisan 2022,2s 22dk,Karanlık büyü Gellert Grindelwald büyü Dünya k...
3,Aykut Enişte 2,"Aykut Enişte 2, sevdiği kadınla evlilik arifes...",['Komedi'],"['Cem Gelinoğlu', 'Melis Babadağ', 'Hakan Yılm...",['Onur Bilgetay'],3 Aralık 2021,1s 55dk,Aykut enişte 2 sevmek kadın ev arife kendi tür...
4,Morbius,"Morbius, hastalığına deva bulmaya çalışırken b...","['Aksiyon', 'Fantastik', 'Macera']","['Jared Leto', 'Matt Smith (XI)', 'Adria Arjona']",['Daniel Espinosa'],1 Nisan 2022,1s 45dk,Morbius hastalık deva bulmak Çalışır bambaşka ...


In [4]:
clean = df["clean"]

In [5]:
len(clean)

5261

In [6]:
import numpy as np

In [7]:
word2idx = {}
idx2word = {}

In [8]:
i = 0
for cell in clean:
    words = cell.split()
    for word in words:
        if word in word2idx:
            continue
        word2idx[word] = i
        idx2word[i] = word
        i += 1

In [9]:
film_sayisi = len(clean)
vocab_size = len(word2idx)

In [10]:
filmler = np.zeros((film_sayisi, vocab_size))

In [11]:
filmler.shape

(5261, 27754)

In [12]:
for i, cell in enumerate(clean):
    words = cell.split()
    for word in words:
        wordIndex = word2idx[word]
        filmler[i][wordIndex] += 1

In [13]:
doc_frequencies = {}

In [14]:
filmler[7][:10]

array([0., 0., 0., 0., 2., 0., 1., 2., 0., 0.])

In [15]:
for word in word2idx:
    doc_count = 0
    for film in clean:
        if word in film:
            doc_count += 1
    
    doc_frequencies[word] = doc_count

In [16]:
sorted_frequencies = sorted(doc_frequencies, key=doc_frequencies.get, reverse=True)

In [17]:
for word in sorted_frequencies[-10:]:
    print(word, doc_frequencies[word])

Daddy 1
menü 1
hindi 1
kaçırıldığında 1
labaratuarla 1
Labaratuarda 1
1621 1
Peluchonneau 1
şairane 1
bitecekmiş 1


In [18]:
filmler_tf_idf = np.zeros((film_sayisi, vocab_size))
film_sayisi = len(clean)

for i, film in enumerate(filmler):
    if i % 1000 == 0:
        print(f"{i} inci film işleniyor...")
    for j, tf in enumerate(film):
        idf = 0
        if tf > 0:
            idf = np.log(film_sayisi / (doc_frequencies[idx2word[j]] + 1)) + 1
        tf_idf_score = tf * idf
        filmler_tf_idf[i][j] = tf_idf_score

0 inci film işleniyor...
1000 inci film işleniyor...
2000 inci film işleniyor...
3000 inci film işleniyor...
4000 inci film işleniyor...
5000 inci film işleniyor...


In [19]:
turler = { 'Dram', 'Biyografik', 'Animasyon','Aile','Macera','Fantastik', 'Komedi', 'Aksiyon','Polisiye', 'Gerilim' ,'Biyografik','Bilimkurgu','Romantik','Erotik','Tarihi', 'Korku','Dramatik komedi','Müzik','Savaş filmi','Casusluk','Bollywood','Western','Müzikal','Epik'}
len(turler)
for k,word2 in enumerate(turler):
  print(word2)

Bollywood
Korku
Dramatik komedi
Epik
Gerilim
Romantik
Animasyon
Fantastik
Western
Aile
Casusluk
Macera
Komedi
Müzikal
Bilimkurgu
Aksiyon
Tarihi
Müzik
Erotik
Dram
Polisiye
Biyografik
Savaş filmi


In [20]:
print(filmler_tf_idf[0][160])

1.8088889632230116


In [21]:
for j in range(5261):
   for i,word1 in enumerate(word2idx):
      for k,word2 in enumerate(turler):
        if(word2 in word1):
          filmler_tf_idf[j][i]=(filmler_tf_idf[j][i])*8

In [22]:
print(filmler_tf_idf[0][160])

14.471111705784093


In [23]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [24]:
def film_2_str(row_index):
    film = df.iloc[row_index]
    adi = film['film adi']
    turu = film['tur']
    ozet = film['ozet ve detaylar']
    if len(ozet) > 150:
        ozet = ozet[:150] + "..."

    return f"Film adı: {adi} \nTürü: {turu} \nÖzeti: {ozet}"

In [25]:
def most_similar_films(film_adi, benzer_film_sayisi):
    film_indexi = df[df["film adi"] == film_adi].index[0]
    print(film_indexi)
    print("Aranan Film:")
    print(film_2_str(film_indexi))
    print()
    print("* " * 30)
    print()

    benzerlikler = []
    sorgu_film = filmler_tf_idf[film_indexi]
    print(sorgu_film)
  
    for vec in filmler_tf_idf:
        benzerlik = cos_sim(sorgu_film, vec)
        benzerlikler.append(benzerlik)
    
    en_benzer_film_indexleri = (- np.array(benzerlikler)).argsort()[:benzer_film_sayisi + 1]
    print("Benzer Filmler:\n")
    for i, film_idx in enumerate(en_benzer_film_indexleri[1:]):
        print(str(i+1) + ") " + film_2_str(film_idx))
        print()
        print("- " * 20)
        print()

In [26]:
most_similar_films("Matrix", 5)

323
Aranan Film:
Film adı: Matrix 
Türü: ['Aksiyon', 'Bilimkurgu'] 
Özeti: Bir yaşanan gerçek vardır, bir de yaşananın ötesinde olan gerçeklik... Biri rüya, diğeri ise Matrix! Neo,son derece tehlikeli bir adam olan Morpheus’u...

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

[0. 0. 0. ... 0. 0. 0.]
Benzer Filmler:

1) Film adı: Matrix Resurrections 
Türü: ['Bilimkurgu', 'Aksiyon'] 
Özeti: Matrix Resurrections”la çifte gerçeklik dünyasına geri dönülüyor. Bay Anderson yeniden beyaz tavşanı izlemeyi seçmek zorunda kalır. Çünkü Anderson ger...

- - - - - - - - - - - - - - - - - - - - 

2) Film adı: Matrix Reloaded 
Türü: ['Bilimkurgu', 'Aksiyon'] 
Özeti: Neo kendisini ajanların ve makinelerin ellerinden kurtarmayı başarmıştır. Ancak makineler insanların elde kalan son şehri olan Zion'u bulma konusunda ...

- - - - - - - - - - - - - - - - - - - - 

3) Film adı: Matrix Revolutions 
Türü: ['Bilimkurgu', 'Aksiyon'] 
Özeti: Makineler Zion'a ulaşmak için kazmaktadırlar. Ajan Smit